In [ ]:
from Solution import *
solve = Solution()
G = solve.build_graph()

In [ ]:
import pandas as pd
data = pd.read_csv('TempResult\社区路口连接数据.csv')
intersection = list(set(data['最近路口节点']))
community = pd.read_excel('Data\community.xlsx')
demand = (community['小区人口数（人）']*0.4).tolist()

In [ ]:
distance = list()
for i in range(len(intersection)):
    start = intersection[i]
    shortest_distance_dict = nx.shortest_path_length(G, source='i'+str(start),weight='weight')
    distance.append([shortest_distance_dict['c'+ str(i)] for i in range(1,1410)])
distance = np.array(distance)

In [ ]:
import gurobipy as gb
intersection_length=len(intersection)
community_length=1409
model=gb.Model()
Q = model.addVars(intersection_length,community_length,lb=0.0,ub=40000,vtype=gb.GRB.CONTINUOUS, name='Q')  # 添加变量
R = model.addVars(intersection_length,vtype=gb.GRB.BINARY,name='R')  # 添加变量
B = model.addVars(intersection_length,vtype=gb.GRB.INTEGER,name='B',lb=0,ub=4)
L = model.addVars(intersection_length,vtype=gb.GRB.INTEGER,name='L',lb=0,ub=10)
print("变量添加完毕")
model.addConstrs(
    gb.quicksum(Q[k,l]*R[k] for k in range(intersection_length)) == demand[l] for l in range(community_length)
)
print('约束1添加完毕')
model.addConstrs(
    gb.quicksum(Q[k,l] for l in range(community_length)) <= 40000 for k in range(intersection_length)
)
print("约束2添加完毕")
model.addConstr(
    gb.quicksum(R[k] for k in range(intersection_length)) <= 71
)
print("约束3添加完毕")
model.addConstrs(
    B[k]*10000 + L[k]*4000 - gb.quicksum(Q[k,l] for l in range(community_length)) >= 0 for k in range(intersection_length)
)
print("约束4添加完毕")

In [ ]:

model.setObjective(
    gb.quicksum(
        Q[k,l] * distance[k,l] * R[k] for l in range(community_length) for k in range(intersection_length)
    ) + 2*gb.quicksum(
        R[k]*B[k] for k in range(intersection_length)
        ) + gb.quicksum(R[k]*L[k] for k in range(intersection_length)), gb.GRB.MINIMIZE
)  # 添加目标函数
print("目标函数设置完毕")

In [ ]:
model.Params.TimeLimit=3600
model.optimize()

In [ ]:
intersection_node = list()
intersection_x = list()
intersection_y = list()
for i in range(len(intersection)):
    if round(R[i].X) != 0:
        intersection_node.append(intersection[i])

In [ ]:
res = pd.DataFrame({'intersection_node':intersection_node})
res.to_csv('有货车.csv',index=False)

In [ ]:
old_data = pd.read_excel('Data\intersection_node.xlsx')
intersection_node = pd.read_csv('有货车.csv')['intersection_node']
intersection_node = pd.merge(intersection_node,old_data,left_on='intersection_node',right_on='节点编号',
                             how='left')[['节点编号','路口横坐标','路口纵坐标']]
areas = pd.read_csv('TempResult\社区路口连接数据.csv')
intersection_node = pd.merge(intersection_node,areas,left_on='节点编号',right_on='最近路口节点',how='left')
intersection_node = intersection_node[['节点编号','路口横坐标','路口纵坐标','所属区域']]
intersection_node.drop_duplicates(subset=['节点编号'], keep='first', inplace=True)
intersection_node.reset_index(drop=True, inplace=True)
intersection_node.to_csv('有货车结果(集散中心).csv',index=False)
intersection_node.to_excel('有货车结果(集散中心).xlsx',index=False)

In [ ]:
groups = intersection_node.groupby(['所属区域'])
areas=list()
center_x = list()
center_y = list()
for i in groups:
    ar,group_data = i
    areas.append(ar)
    center_x.append(group_data['路口横坐标'].mean())
    center_y.append(group_data['路口纵坐标'].mean())
cangku = pd.DataFrame({
    '节点编号':[i+1 for i in range(len(areas))],
    '区域':areas,
    '仓库中心横坐标':center_x,
    '仓库中心纵坐标':center_y
})
cangku.to_csv('有货车结果(仓库).csv',index=False)
cangku.to_excel('有货车结果(仓库).xlsx',index=False)

In [ ]:
from Solution import *
solve = Solution() 
solve.draw_solution_car()